In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
import gc
from datasets import load_dataset, DatasetDict, Dataset
import numpy as np
from sklearn.metrics import balanced_accuracy_score, precision_recall_fscore_support, accuracy_score, classification_report, matthews_corrcoef
import pandas as pd
import wandb
import os

/home/mike/miniforge3/envs/modern/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ["WANDB_PROJECT"] = "DEBATE-ModernBERT-Base"

modname = "MoritzLaurer/ModernBERT-base-zeroshot-v2.0"
training_directory ='training_ModernBase'
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

In [ ]:
ds = load_dataset("mlburnham/Pol_NLI")

In [4]:
tokenizer = AutoTokenizer.from_pretrained(modname)
id2label = {0: "entailment", 1: "not_entailment"}
label2id = {"entailment":0, "not_entailment":1}

def tokenize_function(docs):
    return tokenizer(docs['premise'], docs['augmented_hypothesis'], padding = False, truncation = False)
def model_init():
  return AutoModelForSequenceClassification.from_pretrained(modname, 
                                                           num_labels=2,
                                                           ignore_mismatched_sizes=True,
                                                           label2id = label2id, 
                                                           id2label = id2label)

dstok = ds.map(tokenize_function, batched = True)
dstok = dstok.rename_columns({'entailment':'label'})

training_args = TrainingArguments(output_dir=training_directory,
    logging_dir=f'{training_directory}/logs',
    lr_scheduler_type= "linear",
    group_by_length=True,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=1,
    num_train_epochs=10,
    warmup_ratio=0.06, 
    weight_decay=0.01,
    fp16=True,
    fp16_full_eval=True,
    eval_strategy="epoch",
    seed=1,
    save_strategy="epoch",
    dataloader_num_workers = 1
)

def compute_metrics_standard(eval_pred, label_text_alphabetical=list(id2label.values())):
    labels = eval_pred.label_ids
    pred_logits = eval_pred.predictions
    preds_max = np.argmax(pred_logits, axis=1)

    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(labels, preds_max, average='macro')
    precision_micro, recall_micro, f1_micro, _ = precision_recall_fscore_support(labels, preds_max, average='micro')
    acc_balanced = balanced_accuracy_score(labels, preds_max)
    acc_not_balanced = accuracy_score(labels, preds_max)
    mcc = matthews_corrcoef(labels, preds_max)

    metrics = {'MCC': mcc,
            'f1_macro': f1_macro,
            'f1_micro': f1_micro,
            'accuracy_balanced': acc_balanced,
            'accuracy': acc_not_balanced,
            'precision_macro': precision_macro,
            'recall_macro': recall_macro,
            'precision_micro': precision_micro,
            'recall_micro': recall_micro,
            }
    print("Aggregate metrics: ", {key: metrics[key] for key in metrics if key not in ["label_gold_raw", "label_predicted_raw"]} ) 
    print("Detailed metrics: ", classification_report(
        labels, preds_max, labels=np.sort(pd.factorize(label_text_alphabetical, sort=True)[0]),
        target_names=label_text_alphabetical, sample_weight=None,
        digits=2, output_dict=True, zero_division='warn'),
    "\n")

    return metrics

trainer = Trainer(
    model_init=model_init,
    processing_class=tokenizer,
    args=training_args,
    train_dataset=dstok['train'],
    eval_dataset=dstok['validation'],
    compute_metrics=lambda x: compute_metrics_standard(x, label_text_alphabetical=list(id2label.values()))
)

Device: cuda


You are attempting to use Flash Attention 2.0 without specifying a torch dtype. This might lead to unexpected behaviour
You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.
Flash Attention 2.0 only supports torch.float16 and torch.bfloat16 dtypes, but the current dype in ModernBertForSequenceClassification is torch.float32. You should run training or inference using Automatic Mixed-Precision via the `with torch.autocast(device_type='torch_device'):` decorator, or load the model with the `torch_dtype` argument. Example: `model = AutoModel.from_pretrained("openai/whisper-tiny", attn_implementation="flash_attention_2", torch_dtype=torch.float16)`


In [5]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Mcc,F1 Macro,F1 Micro,Accuracy Balanced,Accuracy,Precision Macro,Recall Macro,Precision Micro,Recall Micro
1,0.181800,0.216367,0.857730,0.928861,0.930567,0.929137,0.930567,0.928594,0.929137,0.930567,0.930567
2,0.102800,0.243936,0.876359,0.938165,0.939612,0.938709,0.939612,0.937650,0.938709,0.939612,0.939612
3,0.051000,0.291324,0.883721,0.941852,0.943336,0.941437,0.943336,0.942285,0.941437,0.943336,0.943336
4,0.021900,0.336943,0.888035,0.944017,0.945398,0.943989,0.945398,0.944046,0.943989,0.945398,0.945398
5,0.014100,0.443264,0.884211,0.942106,0.943536,0.942058,0.943536,0.942153,0.942058,0.943536,0.943536
6,0.011800,0.373732,0.889789,0.944834,0.946329,0.943724,0.946329,0.946067,0.943724,0.946329,0.946329
7,0.007700,0.474585,0.888586,0.944265,0.945730,0.943506,0.945730,0.945081,0.943506,0.945730,0.945730
8,0.001700,0.508356,0.891517,0.945756,0.947060,0.946004,0.947060,0.945514,0.946004,0.947060,0.947060
9,0.001300,0.562439,0.890866,0.945377,0.946595,0.946457,0.946595,0.944412,0.946457,0.946595,0.946595
10,0.000000,0.587830,0.894854,0.947427,0.948723,0.947398,0.948723,0.947456,0.947398,0.948723,0.948723


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


Aggregate metrics:  {'MCC': np.float64(0.8577303790562917), 'f1_macro': 0.9288613150975644, 'f1_micro': 0.9305666400638468, 'accuracy_balanced': np.float64(0.9291367504861167), 'accuracy': 0.9305666400638468, 'precision_macro': 0.9285938004159882, 'recall_macro': 0.9291367504861167, 'precision_micro': 0.9305666400638468, 'recall_micro': 0.9305666400638468}
Detailed metrics:  {'entailment': {'precision': 0.9156853509185116, 'recall': 0.9200189304306673, 'f1-score': 0.9178470254957507, 'support': 6339.0}, 'not_entailment': {'precision': 0.9415022499134649, 'recall': 0.938254570541566, 'f1-score': 0.939875604699378, 'support': 8697.0}, 'accuracy': 0.9305666400638468, 'macro avg': {'precision': 0.9285938004159882, 'recall': 0.9291367504861167, 'f1-score': 0.9288613150975644, 'support': 15036.0}, 'weighted avg': {'precision': 0.9306181502374202, 'recall': 0.9305666400638468, 'f1-score': 0.9305886159010412, 'support': 15036.0}} 



/tmp/ipykernel_24040/497967894.py:92: FutureWarning: factorize with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  labels, preds_max, labels=np.sort(pd.factorize(label_text_alphabetical, sort=True)[0]),
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


...

Aggregate metrics:  {'MCC': np.float64(0.8908658430899224), 'f1_macro': 0.9453771327617819, 'f1_micro': 0.9465948390529396, 'accuracy_balanced': np.float64(0.9464565872081592), 'accuracy': 0.9465948390529396, 'precision_macro': 0.9444116030111819, 'recall_macro': 0.9464565872081592, 'precision_micro': 0.9465948390529396, 'recall_micro': 0.9465948390529396}
Detailed metrics:  {'entailment': {'precision': 0.9290142591444513, 'recall': 0.9455750118315192, 'f1-score': 0.9372214838558361, 'support': 6339.0}, 'not_entailment': {'precision': 0.9598089468779124, 'recall': 0.9473381625847993, 'f1-score': 0.9535327816677276, 'support': 8697.0}, 'accuracy': 0.9465948390529396, 'macro avg': {'precision': 0.9444116030111819, 'recall': 0.9464565872081592, 'f1-score': 0.9453771327617819, 'support': 15036.0}, 'weighted avg': {'precision': 0.9468262702656213, 'recall': 0.9465948390529396, 'f1-score': 0.9466561311736081, 'support': 15036.0}} 



/tmp/ipykernel_24040/497967894.py:92: FutureWarning: factorize with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  labels, preds_max, labels=np.sort(pd.factorize(label_text_alphabetical, sort=True)[0]),
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


Aggregate metrics:  {'MCC': np.float64(0.8948536909586216), 'f1_macro': 0.9474268058867525, 'f1_micro': 0.9487230646448523, 'accuracy_balanced': np.float64(0.9473981024185927), 'accuracy': 0.9487230646448523, 'precision_macro': 0.9474555903866249, 'recall_macro': 0.9473981024185927, 'precision_micro': 0.9487230646448523, 'recall_micro': 0.9487230646448523}
Detailed metrics:  {'entailment': {'precision': 0.9393939393939394, 'recall': 0.938949361097965, 'f1-score': 0.9391715976331361, 'support': 6339.0}, 'not_entailment': {'precision': 0.9555172413793104, 'recall': 0.9558468437392205, 'f1-score': 0.955682014140369, 'support': 8697.0}, 'accuracy': 0.9487230646448523, 'macro avg': {'precision': 0.9474555903866249, 'recall': 0.9473981024185927, 'f1-score': 0.9474268058867525, 'support': 15036.0}, 'weighted avg': {'precision': 0.9487198477051108, 'recall': 0.9487230646448523, 'f1-score': 0.9487214175562143, 'support': 15036.0}} 



/tmp/ipykernel_24040/497967894.py:92: FutureWarning: factorize with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  labels, preds_max, labels=np.sort(pd.factorize(label_text_alphabetical, sort=True)[0]),


TrainOutput(global_step=107060, training_loss=0.045580504342477825, metrics={'train_runtime': 6446.2592, 'train_samples_per_second': 265.718, 'train_steps_per_second': 16.608, 'total_flos': 1.1278615761765728e+17, 'train_loss': 0.045580504342477825, 'epoch': 10.0})